In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings(action="ignore")

In [ ]:
df_train = pd.read_csv('../data/train_cleaned.csv')
df_test = pd.read_csv('../data/test_cleaned.csv')
df_train_b = pd.read_csv('../data/train_cleaned.csv')
df_test_b = pd.read_csv('../data/test_cleaned.csv')

df_train.head()

In [ ]:
df_train = df_train.drop(['Unnamed: 0.1', 'Unnamed: 0', 'title'], axis=1)
df_train.head()

In [ ]:
#Calculate price per sqft of every row
df_train["price_per_sqft"] = df_train["price"]/df_train["size_sqft"] 
df_train["price_per_sqft"] = df_train["price_per_sqft"].round(2)


In [ ]:
#Target encode Property Name
prop_name_agg = df_train.groupby('property_name').agg('mean')
df_train = df_train.merge(prop_name_agg["price_per_sqft"], on=['property_name'], how='left')
df_train.rename(columns={"price_per_sqft_y": "property_name_encoded"}, inplace=True)
df_train.drop(["property_name", "available_unit_types"], axis=1, inplace=True)
df_train.head()


In [ ]:
#Target encode Address

address_agg = df_train.groupby('address').agg('mean')
df_train = df_train.merge(address_agg["price_per_sqft_x"], on=['address'], how='left')
df_train.rename(columns={"price_per_sqft_x_x": "price_per_sqft"}, inplace=True)
df_train.rename(columns={"price_per_sqft_x_y": "address_encoded"}, inplace=True)
df_train.drop(["address"], axis=1, inplace=True)



In [ ]:
#Target encode Subzone and drop planning Area
subzone_agg = df_train.groupby('subzone').agg('mean')
df_train = df_train.merge(subzone_agg["price_per_sqft"], on=['subzone'], how='left')
df_train.rename(columns={"price_per_sqft_x": "price_per_sqft"}, inplace=True)
df_train.rename(columns={"price_per_sqft_y": "subzone_encoded"}, inplace=True)
df_train.drop(["subzone", "planning_area"], axis=1, inplace=True)


In [ ]:
#Label Encode property_type
pd.set_option("display.max_columns", 15)
type_to_enum_map = {t: idx+1 for idx, t in enumerate(sorted(df_train.property_type.unique()))}
df_train.property_type = df_train.property_type.replace(type_to_enum_map,).astype(np.int16)
df_train.head()

In [ ]:
#One hot encode furnishing
x = pd.get_dummies(df_train.furnishing)
df_train['furnishing_unspecified'] = x.unspecified.astype(int)
df_train['furnishing_partial'] = x.partial.astype(int)
df_train['furnishing_unfurnished'] = x.unfurnished.astype(int)
df_train['furnishing_fully'] = x.fully.astype(int)
df_train.drop(columns=['furnishing'], axis = 1, inplace=True)

df_train.head()


In [ ]:
#One hot encode tenure

pd.set_option("display.max_columns", 25)

x = pd.get_dummies(df_train.tenure)
#print(x)
df_train['tenure_99'] = x['99-year leasehold'].astype(int)
df_train['tenure_999'] = x['999-year leasehold'].astype(int)
df_train['tenure_freehold'] = x['freehold'].astype(int)
df_train.drop(columns=['tenure', 'lat', 'lng'], axis = 1, inplace=True)

In [ ]:
#Built Year
df_train.built_year = 2022 - df_train.built_year
df_train.head()


In [ ]:
df_train.to_csv('../data/train_encoded.csv')